In [1]:
import math,random
from matplotlib import pyplot as plt

#clustering
#需要第四章的線性代數函數

class KMeans:
    def __init__(self,k):
        self.k=k
        self.means=None
        
    def classify(self,input):
        return min(range(self.k),
                  key=lambda i:squared_distance(input,self.means[i]))
    
    def train(self,inputs):
        self.means=random.sample(inputs,self.k)
        assignments=None
        
        while True:
            new_assignments=map(self.classify,inputs)
            
            if assignments==new_assignments:
                return
            assignments=new_assignments
            
            for i in range(self.k):
                i_points=[p for p, a in zap(inputs,assignments) if a==1]
                
                if i_points:
                    self.means[i]=vector_mean(i_points)


In [2]:
random.seed(0)
clusterer=KMeans(3)
clusterer.train(inputs)
print clusterer.means


def squared_clustering_errors(inputs,k):
    clusterer=KMeans(k)
    clusterer.train(inputs)
    means=clusterer.means
    assignments=map(clusterer.classify,inputs)
    
    return sum(squared_distance(input,means[cluster])
              for input,cluster in zip(inputs,assignments))

ks=range(1,len(inputs)+1)
errors=[squared_clustering_errors(inputs,k) for k in ks]

plt.plot(ks,errors)
plt.xticks(ks)
plt.xlabel("k")
plt.ylabel("total squared error")
plt.title("Total Error vs. # of Clusters")
plt.show()


NameError: name 'inputs' is not defined

In [3]:
path_to_png_file=r"C:\images\image.png"
import matplotlib.image as mpimg
img=mpimg.imread(path_to_png_file)

top_row=img[0]
top_left_pixel=top_row[0]
red,green,blue=top_left_pixel

pixels=[pixel for row in img for pixel in row]

clusterer=KMeans(5)
clusterer.train(pixels)

def recolor(pixel):
    cluster=clusterer.classify(pixel)
    return clusterer.means[cluster]

new_img=[[recolor(pixel) for pixel in row]
        for row in img]

IOError: [Errno 2] No such file or directory: 'C:\\images\\image.png'

In [ ]:
#bottom-up clustering

leaf1=([10,20],)
leaf2=([30,-15],)

merged=(1,[leaf1,leaf2])

def is_leaf(cluster):
    return len(cluster)==1

def get_childern(cluster):
    if is_leaf(cluster):
        raise TypeError("a leaf cluster has no childern")
    else:
        return cluster[1]

def get_values(cluster):
    if is_leaf(cluster):
        return cluster
    
    else:
        return [value for child in get_children(cluster)
               for value in get_values(child)]


def cluster_distance(cluster1,cluster2,distance_agg=min):
    return distance_agg([distance(input1,input2)
                        for input1 in get_values(cluster1)
                        for input2 in get_values(cluster2)])

def get_merge_order(cluster):
    if is_leaf(cluster):
        return float('inf')
    else:
        return cluster[0]

def bottom_up_cluster(inputs,distance_agg=min):
    clusters=[(input,) for input in inputs]
    
    while len(clusters)>1:
        c1,c2=min([(cluster1,cluster2)
                 for i, cluster1 in enumerate(clusters)
                 for cluster2 in clusters[:i]],
                 key=lambda(x,y):cluster_distance(x,y,distance_agg))
        clusters=[c for c in clusters if c != c1 and c!= c2]
        
        merged_cluster=(len(clusters),[c1,c2])
        cluster.append(merged_cluster)
    return clusters[0]

#use it
base_cluster=bottom_up_cluster(inputs)


def generate_clusters(base_cluster,num_clusters):
    clusters=[base_cluster]
    
    while len(clusters)<num_clusters:
        next_cluster=min(clusters,key=get_merge_order)
        
        clusters=[c for c in clusters if c != next_cluster]
        clusters.extend(get_children(next_cluster))
    return clusters

three_clusters=[get_values(cluster)
               for cluster in generate_clusters(base_cluster,3)]

#ploting
for i, cluster,marker,color in zip([1,2,3],
                                  three_clusters,
                                  ['D','o','*'],
                                  ['r','g','b']):
    xs,ys=zip(*cluster)
    plt.scatter(xs,ys,color=color,marker=marker)
    
    x,y=vector_mean(cluster)
    plt.plot(x,y,marker='$'+str(i)+'$',color='black')
    
plt.title("User Locations -- 3 Bottom-Up Clusters,Min")
plt.xlabel("blocks east of city center")
plt.ylabel("blocks north of cilty center")
plt.show()

